In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop 

In [3]:
path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [4]:
text = open(path, 'rb').read().decode(encoding='utf-8')

In [5]:
charcters = sorted(set(text))
charcters_indices = dict((c, i) for i, c in enumerate(charcters))
indices_charcters = dict((i, c) for i, c in enumerate(charcters))
print(charcters_indices)
print(indices_charcters)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [6]:
seq_len =  40
stp_size = 3

sentences = []
next_chars = []

for i in range(0, len(text) - seq_len, stp_size):
    sentences.append(text[i: i + seq_len])
    next_chars.append(text[i + seq_len])

In [7]:

x = np.zeros((len(sentences), seq_len, len(charcters)), dtype=bool)
y = np.zeros((len(sentences), len(charcters)), dtype=bool)

In [8]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, charcters_indices[char]] = 1
    y[i, charcters_indices[next_chars[i]]] = 1

## RNN MODEL

In [20]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_len, len(charcters))))
model.add(Dense(len(charcters)))
model.add(Activation('softmax'))

In [21]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01), metrics=['accuracy'])

In [22]:
model.fit(x, y, batch_size=64, epochs=10)

Epoch 1/10
5810/5810 [==============================] - 61s 10ms/step - loss: 1.8760 - accuracy: 0.4580
Epoch 2/10
5810/5810 [==============================] - 59s 10ms/step - loss: 1.6470 - accuracy: 0.5177
Epoch 3/10
5810/5810 [==============================] - 59s 10ms/step - loss: 1.5960 - accuracy: 0.5317
Epoch 4/10
5810/5810 [==============================] - 59s 10ms/step - loss: 1.5717 - accuracy: 0.5388
Epoch 5/10
5810/5810 [==============================] - 60s 10ms/step - loss: 1.5520 - accuracy: 0.5433
Epoch 6/10
5810/5810 [==============================] - 60s 10ms/step - loss: 1.5410 - accuracy: 0.5460
Epoch 7/10
5810/5810 [==============================] - 58s 10ms/step - loss: 1.5358 - accuracy: 0.5480
Epoch 8/10
5810/5810 [==============================] - 58s 10ms/step - loss: 1.5320 - accuracy: 0.5490
Epoch 9/10
5810/5810 [==============================] - 57s 10ms/step - loss: 1.5332 - accuracy: 0.5489
Epoch 10/10
5810/5810 [==============================] - 57s 10m

In [32]:
model.save('shakesper.h5')

## USing model

In [33]:
model = tf.keras.models.load_model('shakesper.h5')

In [34]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [35]:
def genrate_text(length, temperature):
    start_index = random.randint(0, len(text) - seq_len - 1)
    generated = ''
    sentence = text[start_index: start_index + seq_len]
    generated += sentence
    for i in range(length):
        x_pred = np.zeros((1, seq_len, len(charcters)))
        for t, char in enumerate(sentence):
            x_pred[0, t, charcters_indices[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_charcters[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

In [39]:
print("__________________________________________________________")
print(genrate_text(300, 0.76))
print("__________________________________________________________")
print(genrate_text(300, 0.4))
print("__________________________________________________________")
print(genrate_text(300, 0.1))
print("__________________________________________________________")
print(genrate_text(300, 0.3))
print("__________________________________________________________")
print(genrate_text(300, 0.9))


__________________________________________________________
at loves not me, nor none of you.

RIVEREY:
I will thou shall be the man in the straight.

CORIOLANUS:
I will thou show the man, and the man and the fair son of the man.

CORIOLANUS:
I have the man and the man and the man.

CORIOLANUS:
I will not shall be the man and the man.

GLOUCESTER:
The man and the man and the man and the man.

GLOU
__________________________________________________________
en forget himself?

First Huntsman:
Belike, sir, I will she and the man, the fair sick of the comples,
That the brother and the man and the man.

CORIOLANUS:
I would not shall be the many the brother
And the man and the father and the man.

GLOUCESTER:
The man and the man and the brother and the fair son,
And the man and the brother and t
__________________________________________________________
Turks and infidels,
And in this seat of the seast of the grace.

CORIOLANUS:
I have the better and the man and the man.

CORIOLANUS:
I am the